# Artificial Intelligence Course, Dr.Fadaei

# Computer Assignment #2

## AI Hands-on: Genetic

## Parnian fazel (810198516)

### Goal:

In this project we should use the genetic algorithms in order to solve a sudoku game. We get familiar with the **reproduction cycle** and we use **selection**, **mutation**, **crossover** and other **GA operators** to reach this goal.

### Brief Description:

In this assugnment we get a **sudoku** table as input and we should solve this table using the genetic algorithm. Sudoku is played on a grid of 9 x 9 spaces. Within the rows and columns are 9 squares and contains 9  3x3 blockes. Each row, column and block needs to be filled out with the numbers 1-9, without repeating any numbers within the row, column or square. 

### What is GA and How It works?

The genetic algorithm is a random-based classical evolutionary algorithm. By random here we mean that in order to find a solution using the GA, random changes applied to the current solutions to generate new ones. GA is based on Darwin’s theory of evolution. GA works on a population consisting of some solutions where the population size is the number of solutions. Each solution is called individual. Each individual solution has a chromosome. The chromosome is represented as a set of parameters (features) that defines the individual. Each chromosome has a set of genes. Each individual has a fitness value. To select the best individuals, a fitness function is used. The result of the fitness function is the fitness value representing the quality of the solution. The higher the fitness value the higher the quality the solution. Selection of the best individuals based on their quality is applied to generate what is called a mating pool where the higher quality individual has higher probability of being selected in the mating pool. All these opratores are defind ih the algorithm below.

### Libraries ipmorted

In [1]:
import numpy as np
import random
random.seed()
import time

### Hyperparameters

Here I defined the hyperparameters I used through the algorithm. these parameters are very imprtant and changing them would effect the optimality and completeness of the algorithm. I did not define the mutation rate in this part beacuase I used dynamic mutation rate and it is not constant. The mutation rate is defined in the GA class.

In [2]:
N = 9
POPULATION_SIZE = 1000
NUMBER_OF_GENERATIONS = 400
SELECTION_RATE = 0.2
CROSSOVER_RATE = 0.9
LOCAL_OPTIMAL_SIZE = 10
MAX_LOCAL_OPTIMA_NUMBER = 10
INCREASE_MUTATION_RATE = 0.05

### Encoding:

The encoding of individuals (also known as chromosome encoding) is fundamental to the implementation of GAs in order to efficiently transmit the genetic information from parents to offsprings.
* **Gene**: Every cell of the table
* **Chromosome**: Each possible solution table 

### Individual

Here I defined The Individual Class:
This class has the table as the grid attribute which is a 2D array and a fitness value. which will be set during the reproduction cycle.
This class hase several methodes:
* **setFitness()**: This function calculates the fitness of a chromosome and returns it. The fitness function count total number of unique values in each row, column and 3x3 block. So if the fitness value of a chromosome is $81 * 3 = 243$ it is the final solution.
* **getMutationRange()**: This methodes finds all the valid numbers which can be set in a sqaure of a grid. The mutation function use this method in order to find all possible numbers can be used to mutate a square.
* **mutate()**: This function traverse all squares in a grid and mutate each gene if possible (based on the getMutationRange() method described above.

In [3]:
class Individual:
    def __init__(self):
        self.grid = np.zeros((N, N), dtype=int)
        self.fitness = None
        return

    def setFitness(self):
        uniqueNumbers = 0
        tableGrid = np.array(self.grid)
        for i in range(0, N):
            _, count = np.unique(tableGrid[i],  return_counts=True)
            uniqueNumbers += len(count[count == 1])

        for j in range(0, N):
            _, count = np.unique(tableGrid[:,j],  return_counts=True)
            uniqueNumbers += len(count[count == 1])

        for i in range(0, N, 3):
            for j in range(0, N, 3):
                block = tableGrid[i:i+3, j:j+3]
                _, count = np.unique(block,  return_counts=True)
                uniqueNumbers += len(count[count == 1])
        fitness = uniqueNumbers
        self.fitness = fitness
        return fitness


    def getMutationRange(self, i, j):
        availableNumbers = np.arange(1,N+1)
        for c in range(N):
            if (not c == j) and self.grid[i][c] in availableNumbers:
                availableNumbers = availableNumbers[availableNumbers != self.grid[i][c]]
        for r in range(N):
            if (not r == i) and self.grid[r][j] in availableNumbers:
                availableNumbers = availableNumbers[availableNumbers !=  self.grid[r][j]]
        rBlock = (int(i / 3)) * 3
        cBlock = (int(j / 3)) * 3
        for r in range(rBlock, rBlock + 2):
            for c in range(cBlock, cBlock + 2):
                if (not r == i) and (not c == j) and self.grid[r][c] in availableNumbers:
                    availableNumbers = availableNumbers[availableNumbers != self.grid[r][c]]
        return availableNumbers

    def mutate(self, problem):
        for i in range(N):
            for j in range(N):
                if problem.grid[i][j] == 0:
                    availableNumbers = self.getMutationRange(i, j)
                    if len(availableNumbers):
                        chosenValue = random.choice(availableNumbers)
                        self.grid[i][j] = chosenValue



## Population

Here I defined the population class which has list of all chromosemes. This class hase several methodes:
* **generateChromosome():** This method generate a chromosome which the predifined sqaures of the given sudoku are in their place but the empty sqaures will be set by a random value and finally a chromosome is generated. This method is used in generatePopulation() method in the class.
* **generatePopulation():** this function generate chromosomes by employing the `generateChromosome()` method and the number of this chromosomes are defined as hyperparameter `POPULATION_SIZE`.
* **setFitness():** This method traverse through all chromosomes and call `setFitness()` function for each individual in the population

In [4]:
class Population:
    def __init__(self):
        self.chromosomes = []

    def generateChromosome(self, inputBoard):
        chromosome = []
        for i in range(N):
            valuesRange = np.arange(1,N+1)
            chromosome.append(list(inputBoard[i]))
            for j in range(N):
                if chromosome[i][j] == 0:
                    chosenValue = random.choice(valuesRange)
                    chromosome[i][j] = chosenValue
        ch = Individual()
        ch.grid = chromosome
        return ch

    def generatePopulation(self, inputBoard):
        for _ in range (POPULATION_SIZE):
            self.chromosomes.append(self.generateChromosome(inputBoard))

    def setFitness(self):
        for chromosome in self.chromosomes:
            chromosome.setFitness()
        return

    def sort(self):
        self.chromosomes.sort(key=lambda x: x.fitness)
        return

### Genetic Algorithm

In this part I defined the Genetic Algorithm class. This class has several attributes and methodes.

#### Attributes:

* **grid**: it is the given sudoku table as input of the algorithm
* **problem**: it is a individual which its grid is the given sudoku table as input
* **population**: it it the current population which the cycle is running on. At first it is the population of the first generation.
* **numberOfTimesGotInLocalMaxima:** In order to avoid getting in local optima I defined this attribute. every time we get into local optima this varibale increase by 1 and if its value get higher than hyperparameter `MAX_LOCAL_OPTIMA_NUMBER` th cycle will start over.
* **bestIndividuals**: I store the top fitness of chromosomes in each population and if the last `LOCAL_OPTIMAL_SIZE` number of this array are similar means we got into a local optima. I use this attribute in `isInLocalMaxima()` method.

#### Methodes:

* **runReproductionCycle():** This is the main function of the algorithm. I run the reproduction cycle and use all GA operatores in order to get the solution. First I traverse the chromosomes in the current generation and check whther the final solution is found or not meanwhile I store the best fitness to save its value in `bestIndividuals` attribute in order to use it in `isInLocalMaxima()` method. Then I start the evolution part. I store the best chromosomes as they transfer directly to next generation without being crossovered and mutated. The number of this chromosomes is based on the hyperparameter `SELECTION_RATE`. The remaining number of chromosomes in next generation are generated by applying cross over and mutation on the chromosomes of current genearion and also the hyperparameters `CROSSOVER_RATE` and `mutationRate` are considered. Note that the population size does not change in each generaion and it is always as size of hyperparameter `POPULATION_SIZE`.
* **tournamentSelection():** I used 4-way tournamen selection as selection operator. Because this method is fast and keeps the diversity of the selected individuals as the parents of next generation individuals. In this method I select 4 random individuals and return the best of them(which has a better fitness).
* **rankBasedSelection():** I implemented the rank-based selection too, but beacuse of the fact that it was time consuming and in some cases would suffer from convergence, I did not use it.
* **crossover():** This method traverse all N=9 rows of the table and apply 1-point crossover for each row. note that in the reproduction cycle we apply crossover operator by hyperparameter `CROSSOVER_RATE`.
* **onePointCrossOverInRow():** This function implement the 1-point crossover for each row. It generates a random number(`crossPoint`) and swap values according to the 1-point crossover method.
* **mutate():** In this method I apply mutation for a table base on the `mutationRate` variable in the code. This rate is caldulated by $1 - \dfrac {fitness} {3 * N * N}$. It shows how far we are from the final solution. the furthure we are the higher possibility of the mutation will be. Note that here according to the fitness function (described in individual part) the maximum number of fitness for a chromosome is $3 * N * N$.
* **isInLocalMaxima()**: In this method I check wether we are in local optima or not. In order to do this I store best fitness of each generation in a list called `bestIndividuals` and if the last `LOCAL_OPTIMAL_SIZE` number of this list are the same meanes that we are in local optima. In this case I increase the mutation rate by `INCREASE_MUTATION_RATE`. If after `MAX_LOCAL_OPTIMA_NUMBER` number we are still in local optima the reproduction cycle starts over.

In [5]:
class GA:

    def __init__(self):
        self.grid = readFile()
        self.problem = Individual()
        self.problem.grid = self.grid
        self.population = Population()
        self.population.generatePopulation(self.grid)
        self.numberOfTimesGotInLocalMaxima = 0
        self.population.setFitness()
        self.bestIndividuals = []

    def runReproductionCycle(self):
        random.seed()
        for generation in range(NUMBER_OF_GENERATIONS):
            bestFitness = 0
            for i in range(POPULATION_SIZE):
                fitness = self.population.chromosomes[i].fitness
                if fitness == 3 * N * N:
                    print(f"Solution has been found in {generation} generations")
                    print("Solution Grid:")
                    print(self.population.chromosomes[i].grid)
                    return self.population.chromosomes[i]
                if fitness > bestFitness:
                    bestFitness = fitness

            self.bestIndividuals.append(bestFitness)
            nextPopulation = []

            self.population.sort()
            for elite in range(0, int(SELECTION_RATE * POPULATION_SIZE)):
                best = Individual()
                best.grid = np.copy(self.population.chromosomes[int(SELECTION_RATE * POPULATION_SIZE) - elite].grid)
                nextPopulation.append(best)

            for _ in range(int(SELECTION_RATE * POPULATION_SIZE), POPULATION_SIZE):
                parent1 = self.tournamentSelection(self.population.chromosomes)
                parent2 = self.tournamentSelection(self.population.chromosomes)
                child1 = parent1
                child2 = parent2
                if random.random() < CROSSOVER_RATE:
                    child1, child2 = self.crossover(parent1, parent2)
                    child1.setFitness()
                    child2.setFitness()
                if self.numberOfTimesGotInLocalMaxima > MAX_LOCAL_OPTIMA_NUMBER:
                    return None
                child1, child2 = self.mutate(child1, child2)
                nextPopulation.append(child1)
                nextPopulation.append(child2)

            self.population.chromosomes = nextPopulation
            self.population.setFitness()

    def mutate(self, child1, child2):
        mutationRate = 1-(child1.fitness/(3 * N * N))
        if self.isInLocalMaxima(self.bestIndividuals):
            mutationRate += INCREASE_MUTATION_RATE
            self.numberOfTimesGotInLocalMaxima += 1
        if random.random() < mutationRate:
            child1.mutate(self.problem)
            child1.setFitness()
        mutationRate = 1-(child2.fitness/(3 * N * N))
        if self.isInLocalMaxima(self.bestIndividuals):
            mutationRate += INCREASE_MUTATION_RATE
        if random.random() < mutationRate:
            child2.mutate(self.problem)
            child2.setFitness()
        return child1, child2
        
    def crossover(self, parent1, parent2):
        child1 = Individual()
        child2 = Individual()
        child1.grid = np.copy(parent1.grid)
        child2.grid = np.copy(parent2.grid)
        for i in range(N):
            child1.grid[i], child2.grid[i] = self.onePointCrossOverInRow(child1.grid[i], child2.grid[i])
        return child1, child2

    def onePointCrossOverInRow(self, row1, row2):
        crossPoint = random.randint(1,8)
        child1 = [0] * N
        child2 = [0] * N
        for i in range(crossPoint):
            child1[i] = row1[i]
        for i in range(crossPoint, N):
            child1[i] = row2[i]
        for i in range(crossPoint):
            child2[i] = row2[i]
        for i in range(crossPoint, N):
            child2[i] = row1[i]
        return child1, child2

    def tournamentSelection(self, chromosomes):
        c1 = chromosomes[random.randint(0, len(chromosomes)-1)]
        c2 = chromosomes[random.randint(0, len(chromosomes)-1)]
        c3 = chromosomes[random.randint(0, len(chromosomes)-1)]
        c4 = chromosomes[random.randint(0, len(chromosomes)-1)]
        c = [c1, c2, c3, c4]
        f = [c1.fitness, c2.fitness, c3.fitness, c4.fitness]
        idx = np.argmax(f)
        return c[idx]

    def rankBasedSelection(self, chromosomes): 
        summ = sum(range(1,POPULATION_SIZE+1)) 
        r = random.randint(1, summ)
        stemp = 0
        for i in range(len(chromosomes)):
            if stemp >= r:
                return chromosomes[i]
            stemp += i

    def isInLocalMaxima(self, bestIndividuals):
        if len(bestIndividuals) < LOCAL_OPTIMAL_SIZE:
            return False
        recentIndividuals = bestIndividuals[len(bestIndividuals) - LOCAL_OPTIMAL_SIZE:]
        return recentIndividuals[1:] == recentIndividuals[:-1]


### Run Test1

Here I read the input sudoku table as input data.

In [6]:
def readFile():
    f = open("Test1.txt", "r")
    file_lines = f.read().split('\n')
    grid = [[] for i in range(N)]
    for i in range(N):
        line_values = [(int(value) if value != '0' else 0) for value in file_lines[i].split(' ')]
        for j in range(N):
            grid[i].append(line_values[j])
    print("Initial Sudoku Grid: ")
    print(np.matrix(grid))
    return grid

Now I run the reproduction cycle and print the final solution.

In [7]:
ga = GA()
start = time.time()
while (ga.runReproductionCycle() == None):
    print("Got In Local Optima!\nStarting The Reproduction Cycle Again.")
    ga = GA()
    ga.runReproductionCycle()
end = time.time()
print(f"executation time: {end-start}")

Initial Sudoku Grid: 
[[8 0 6 0 0 0 1 0 7]
 [0 0 0 6 0 2 0 0 0]
 [0 5 3 0 0 4 8 0 6]
 [7 0 4 8 0 0 6 3 0]
 [0 0 0 0 0 0 0 9 0]
 [1 0 0 5 0 0 4 0 0]
 [0 0 1 2 0 0 7 0 9]
 [2 0 0 0 9 6 0 0 0]
 [0 7 0 0 1 0 0 8 0]]
Solution has been found in 21 generations
Solution Grid:
[[8 2 6 9 3 5 1 4 7]
 [4 1 7 6 8 2 9 5 3]
 [9 5 3 1 7 4 8 2 6]
 [7 9 4 8 2 1 6 3 5]
 [5 6 8 3 4 7 2 9 1]
 [1 3 2 5 6 9 4 7 8]
 [3 4 1 2 5 8 7 6 9]
 [2 8 5 7 9 6 3 1 4]
 [6 7 9 4 1 3 5 8 2]]
executation time: 147.87599420547485


### Run Test2

In [10]:
def readFile():
    f = open("Test2.txt", "r")
    file_lines = f.read().split('\n')
    grid = [[] for i in range(N)]
    for i in range(N):
        line_values = [(int(value) if value != '0' else 0) for value in file_lines[i].split(' ')]
        for j in range(N):
            grid[i].append(line_values[j])
    print("Initial Sudoku Grid: ")
    print(np.matrix(grid))
    return grid

In [11]:
ga = GA()
start = time.time()
while (ga.runReproductionCycle() == None):
    print("Got In Local Optima!\nStarting The Reproduction Cycle Again.")
    ga = GA()
    ga.runReproductionCycle()
end = time.time()
print(f"executation time: {end-start}")

Initial Sudoku Grid: 
[[0 6 0 2 0 0 0 7 1]
 [4 0 5 0 0 0 0 0 2]
 [3 0 0 0 8 0 6 9 0]
 [2 0 0 9 0 8 7 0 0]
 [0 9 3 0 0 0 8 0 0]
 [0 0 6 0 0 1 0 0 9]
 [0 8 7 0 3 0 0 0 6]
 [6 0 0 0 0 0 5 0 7]
 [0 0 0 0 0 9 0 2 0]]
Solution has been found in 16 generations
Solution Grid:
[[9 6 8 2 5 3 4 7 1]
 [4 7 5 1 9 6 3 8 2]
 [3 1 2 4 8 7 6 9 5]
 [2 5 1 9 4 8 7 6 3]
 [7 9 3 6 2 5 8 1 4]
 [8 4 6 3 7 1 2 5 9]
 [1 8 7 5 3 2 9 4 6]
 [6 2 9 8 1 4 5 3 7]
 [5 3 4 7 6 9 1 2 8]]
executation time: 54.94129753112793


### Questions

### 1. **Explain the selection method and your reason for choosing this method**.

I implemeted 2 selection methods: 1- `Tournament Selection` 2- `Rand-based Selection`. I employed the **4-way Tournament Selection** method because it was more **efficient** and **optimal** and also the **diversity** of the selections was acceptable but **Rank-based Selection** method was **time consuming** and in some cases would suffer from **convergence** so I did not use it. More details are in the code and explained in GA part.

### 2. Explain The reason of choosing fitness function.

The fitness function is the total number of unique values in each row, column and 3x3 block. So if the fitness value of a chromosome is $81 * 3 = 243$ it is the final solution and we have reached the answer, because actually we are considering each gene(sqaue) 3 times. This function is very accurate according to the sudoku rules beacuse we are considering number of different sqauares which pass all 3 conditions of the game rules. Also this function is simple and does not add much time and space complexity.

### 3. what is the effect of crossover and mutation functions? Also explain the mutation rate and crossover rate.

The **crossover** and **mutation** operators are the most important ingredient of GAs. In **crossover**, by selecting individuals from the parental generation and interchanging their genes, new individuals are obtained. Hence, employing a good selection method is vital otherwise it is possible that our algotirhm does not work or not be optimal. In muation function, unlike crossover method, we do not need two parents. Changes are made in genes according to the mutation rate. As the **crossover** function by applying good mutation operators we can generate better offsprings for the next generation therefore our algorith, would be optimal. Although sometimes the mutation operator is discarded and considered as a secondary operator, I used it in this project. Chhosing good combination of mutation and crossover operators can result in improving the performance and completeness of GAs.
Mostly the **crossover rate** ($P_c$) is not small beacause of the fact that we are trying to generate offspring from $2$ parents and evolve the generation so I considered the crossover rate as $0.9$. By choosing a small crossover rate $P_c$ each generation would evolve less so we get far from the main purpuse of using GA. Although these hyperparameters are very dependent on the problem. Sometimes the **mutation rate** ($P_m$) are defined as **constant** but sometimes they are calculated **dynamically** throught the algorithm. In this assignment I used **dynamic mutation rate** based on the situation of the chromosoms and as we get closer to the solution the mutation rate decreses. This rate is caldulated by $1 - \dfrac {fitness} {3 * N * N}$. It shows how far we are from the final solution. Actually the concept of choosing this dynamic mutation rate is like `Simulated Annealing Search`, because as we get closer to the solution the mutation rate decreses and our chromosome would change with less probability. The furthure we are the higher probability of the mutation will be. By applying good mutation rate the diversity of the **pupulation** is kept so $P_m$ can effect on **convergence** of the GA. Usually, large mutation rate would make the GA search to resemble a random search.

### 4. Explain the reasons and problems of getting in a local optima. Suggest some solutions.

In GAs one critical aspect is keeping diversity of the population and avoid premature convergence to local optima.
**Main Reasons:**
* **small diversity:** The main reason is small diversity in the population because sometimes some solutions in population are not good but some part of them could help us to get closer to the solution. and when the chromosomes are like eachother we can not generate different and better offsprings so we get in a local optima. So **diversity** is vital in GAs.
* **small population size:** when the population size is small we are not provided with wide range of solution to the problem so we may lost solutions which can gets us to the final solution. Hence, we get into a local optima.
* **choosing not proper sleection algorithm**: selection operator in GAs are very important. some selection methodes would get us into local optima because they just want to choose the best individuals so after some generations all chromosomes may be more or less the same as eachother. for instance `roulette wheel` selection method is among methodes which would end in local optima but in `tournament` selection is less likely to get into local optima. 
* **Problem:** By getting into a local optima we can not get the best final solution so the GA will not be **complete**.
* **Solutions:** 
* **Increasing population size**: By increasing the population size our choices in each generaion for selecting parent of the next geneartion will get wider so it is less likely to get into a local optima.
* **Increase mutation rate**: When we get into a local optima we can increase mutation rate inorder to get individual to get close to a different local minimum. This can help us get out of a local minimum to finally get to the global optima.

### Concolusion

In this assinmnet I learned the concept of genetic algorithm and how to make them optimal based on the problem. I learned the difference between mutation methodes, selection methodes and crossover methodes and also how they can affect when they are combines effectively. Also I observed how hyperparameters can effect on the algorithm. Now I can employ GAs and choose selection, crossover and mutation methode base on the problem and choose fittest hyperparameters for my solution.

#### References

1- Refernce book of the course: *Artificial Intelligence A Modern Approach*

2- towardsdatascience.com which you can find here: https://towardsdatascience.com/introduction-to-optimization-with-genetic-algorithm-2f5001d9964b

3- Performance evaluation of WMN-GA for different mutation and crossover rates considering number of covered users parameter - Admir Barolli, Fatos Xhafa, Leonard Barolli, Makoto IkedadandMakoto Takizawa